In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf

2025-04-15 14:14:19.974581: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-15 14:14:20.000479: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv('data.csv')
df.head()

,Player,Season,Passing Yds,Passing Tds,Rushing Yds,Rushing Tds,Receiving Yds,Receiving Tds,VBD
0,Ron Johnson,1970,0,0,1027,8,48,5,135
1,Gene Washington,1970,0,0,0,0,44,1,33
2,MacArthur Lane,1970,0,0,977,11,32,2,126
3,Warren Wells,1970,0,0,34,0,43,0,112
4,John Brodie,1970,2941,24,29,2,0,1,105


In [3]:
aggregated = df.groupby("Player").agg({
    'Passing Yds': 'sum',
    'Passing Tds': 'sum',
    'Rushing Yds': 'sum',
    'Rushing Tds': 'sum',
    'Receiving Yds': 'sum',
    'Receiving Tds': 'sum',
    'VBD': 'mean'  # Keep target (VBD) as the mean for regression
}).reset_index()

In [4]:
# Features to normalize
features = ['Passing Yds', 'Passing Tds', 'Rushing Yds', 'Rushing Tds', 'Receiving Yds', 'Receiving Tds']

scaler = MinMaxScaler()
aggregated[features] = scaler.fit_transform(aggregated[features])

In [5]:
X = aggregated.drop(['Player', 'VBD'], axis=1)
y = aggregated['VBD']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # regression output
])

/home/ayush/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=50, batch_size=8, verbose=0)

In [8]:
y_pred = model.predict(X_test).flatten()

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)

print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")

43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
MSE: 30.82
RMSE: 5.55
MAE: 2.43


In [9]:
aggregated["NN Predicted Score"] = model.predict(X).round(2)

top10 = aggregated[['Player', 'NN Predicted Score', 'VBD']].sort_values('NN Predicted Score', ascending=False).head(10)
print('\nTop 10 Players by Predicted Score:')
print(top10)

214/214 ━━━━━━━━━━━━━━━━━━━━ 0s 851us/step

Top 10 Players by Predicted Score:
                   Player  NN Predicted Score         VBD
2172         Emmitt Smith          109.120003   86.933333
4024  LaDainian Tomlinson           96.160004  118.181818
6643        Walter Payton           90.580002   98.846154
4337         Marcus Allen           86.110001   54.937500
4450       Marshall Faulk           79.269997   96.083333
372         Barry Sanders           73.870003  120.700000
2282        Franco Harris           71.739998   45.461538
2185       Eric Dickerson           71.419998   74.454545
3446         John Riggins           70.480003   33.571429
2286           Frank Gore           70.010002   36.250000


In [10]:
aggregated.to_csv('nn.csv', index=False, columns=['Player', 'NN Predicted Score'])